### assetAllocation v4

- python version: 3.8.18

> 03.25 교수님 피드백 적용
- HHI 지수 추가
- volatility 연간으로 계산 후 평균화


In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px

In [2]:
tickers = ['IGLB', 'HYG', 'SPY', 'MBB','BOTZ','ICLN','PAVE']

pxclose = pd.DataFrame()

for t in tickers:
    pxclose[t] = yf.download(t, start = "2010-01-01")['Adj Close']

""" pxclose matrix의 최근 자료들 열람한다."""
pxclose.head()

for t in tickers:
    start_price_spy = pxclose[eval(f'{t}')][0]
    end_price_spy = pxclose[eval(f'{t}')][-1]
    total_return_spy = round((end_price_spy - start_price_spy) / start_price_spy, 2)
    num_years_spy = len(pxclose[t]) / 252 # 1년은 주식 시장의 거래일인 252일로 가정
    average_annual_return_spy = round((1 + total_return_spy) ** (1 / num_years_spy) - 1, 2)
    pxclose[t] = average_annual_return_spy


""" pxclose matrix의 최근 자료들 열람한다."""
pxclose.head()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


NameError: name 'IGLB' is not defined

In [ ]:
# 일간수익률의 1년 중 시장이 열리는 일수를 곱해 기대수익률을 만든다.
ret_daily = pxclose.pct_change() # 일간수익률

yearly_count = ret_daily.resample('Y').count() # 각 연도별 데이터 포인트 개수

ret_annual = ret_daily.resample('Y').mean()

for year in yearly_count.index.year:
    count = yearly_count.loc[str(year)].max() # 해당 연도의 데이터 포인트 개수
    # 해당 연도의 데이터에 개수를 곱함
    ret_annual.loc[ret_annual.index.year == year] *= count

ret_annual_mean = ret_annual.mean()
ret_annual_mean

C:\Users\82108\AppData\Local\Temp\ipykernel_20488\3191368505.py:4: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

C:\Users\82108\AppData\Local\Temp\ipykernel_20488\3191368505.py:6: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



IGLB    0.045722
HYG     0.049059
SPY     0.133637
MBB     0.015066
BOTZ    0.115171
ICLN    0.023406
PAVE    0.156595
dtype: float64

In [ ]:
# 연도별로 데이터 분리 및 연도별 공분산 계산
annual_cov_list = []
for year, group in ret_daily.groupby(ret_daily.index.year):
    # 각 컬럼(티커)의 데이터 개수 계산
    counts = group.count()

    # 공분산 계산
    cov_matrix = group.cov()

    # 각 컬럼의 데이터 개수가 0인 경우 해당 컬럼의 공분산 값을 NaN으로 설정
    cov_matrix = cov_matrix.multiply(counts, axis='columns').multiply(counts, axis='index')
    cov_matrix = np.where(counts == 0, np.nan, cov_matrix)
    
    # 수정된 공분산 행렬을 리스트에 추가
    annual_cov_list.append(pd.DataFrame(cov_matrix, index=group.columns, columns=group.columns))
    #print(pd.DataFrame(cov_matrix)) # for debugging

# 연도별 공분산의 평균 계산
mean_cov_matrix = pd.concat(annual_cov_list).groupby(level=0).mean()

print(mean_cov_matrix)

          IGLB       HYG       SPY       MBB       BOTZ       ICLN       PAVE
BOTZ  2.003112  3.428431  8.543526  0.608045  13.683727  10.293305  10.966728
HYG   0.702292  1.783757  2.739217  0.225566   3.428431   3.477380   3.618466
ICLN  0.799200  3.477380  7.975707  0.333123  10.293305  17.738215  10.317582
IGLB  3.240094  0.702292  0.407007  0.807178   2.003112   0.799200   1.554880
MBB   0.807178  0.225566  0.114462  0.434913   0.608045   0.333123   0.399639
PAVE  1.554880  3.618466  9.018902  0.399639  10.966728  10.317582  13.931328
SPY   0.407007  2.739217  7.083442  0.114462   8.543526   7.975707   9.018902


In [ ]:
# portfolio 수익률, 변동성, 투자 비중
p_returns = []
p_volatility = []
p_weights = []

# len() 함수로 투자자산의 수를 계산한다. 
n_assets = len(tickers)

# n개 종목으로 투자 비중을 바꿔가며 3만 개의 포트폴리오를 만들 것이
n_ports = 30000

"""
# n_ports 만큼 반복하면서 자신의 투자 비중을 랜덤하게 만들고, 포트폴리오의 기대수익률, 변동성을 계산
# 계산한 수익률, 변동성, 투자 비중은 앞서 미리 준비한 변수들인
p_returns, p_volatility, p_weights에 저장한다.
"""

for s in range(n_ports):
    
    #np.random.random() 함수로 난수 생성
    wgt=np.random.random(n_assets)
    #print(f"weights before: {wgt}")
    
    #투자 비중 합계 100%를 위해 각 난수를 난수 합으로 나눈다
    wgt /= np.sum(wgt)
    #print(f"weights after: {wgt}")
    
    """
    # portfolio 기대수익률 = 비중 * 각 종목별 기대수익률
    # dot product로 계산해 준다. 
    여기에서 wgt 는 (n,1) vector이며, ret_annual도 (n,1) vector
    """
    ret = np.dot(wgt, ret_annual_mean)
    
    #portfolio의 변동성을 계산한다. (wgt x covariance matrix x wgt)^(1/2)
    vol = np.sqrt(np.dot(wgt.T, np.dot(mean_cov_matrix,wgt)))
    
    #계산한 수익률, 변동성, 비중을 추가한다. 
    p_returns.append(ret)
    p_volatility.append(vol)
    p_weights.append(wgt)

HHI_threshold = 3000

print(f"before: {len(p_weights)}")

for i in range(len(p_weights) - 1, -1, -1):  # 리스트를 역순으로 순회
    if ((p_weights[i] * 100) ** 2).sum() > HHI_threshold:
        del p_returns[i], p_volatility[i], p_weights[i]

print(f"after: {len(p_weights)}")

# HHI 기준을 충족 포트폴리오 개수
modif_n_ports = len(p_weights)

before: 30000
after: 29843


In [ ]:
"""
# numpy의 random.random(n) 함수는 수학적 의미에서 [0,1] 
구간에 속하는 난수를 n개 만큼 출력한다
# 예를 들어, 
array([0.61240447, 0.12133946, 0.58294007, 0.33440155, 0.11916194])
"""
np.random.random(5)

"""
# wgt / = np.sum(wgt) 부분: 
# /= 왼쪽 변수에 오른쪽 값을 나누고 그 결과를 왼쪽 변수에 할당한다
"""

# portfolio volaility 에서 np.dot은 dot product를 의미 
# wgt.T 는 wgt의 transpose matrix를 의미 
# np.sqrt(a)는 a^(1/2)을 의미 

# np.array를 사용하여 matrix로 인식시킨다
p_volatility = np.array(p_volatility).round(2)
p_returns = np.array(p_returns)

"""
# 색상을 n_ports 만큼 만든다
# random.randint(0, 5)는 수학적 의미에서 [0,5] 구간에 
위치한 정수(integer) 중 하나를 random하게 추출한다
# random.rnadint(0,5,100) 에서 100은 100개를 추출하라는 뜻이다. 
"""

colors=np.random.randint(0, modif_n_ports, modif_n_ports)

fig = px.scatter(x=p_volatility, y=p_returns, color=colors, hover_data=[p_weights])
fig.update_layout(
    title="Efficient Frontier for portfolio('IGLB(장기투자채권)', 'HYG(고수익채권)', 'SPY(S&P500)', 'MBB(모기지)','BOTZ(로봇공학, 인공지능)','ICLN(에너지)','PAVE(인프라')",
    xaxis_title="volatility",
    yaxis_title="returns"
)
fig.show()

NameError: name 'p_volatility' is not defined

<할일>

포트폴리오 도출 기준
- 긱 포트폴리오의 HHI값 구하기(well diversified를 위해)
    - threshold를 지정해줘야 함.(총 10000 중에 어떤 값인가) - if 숫자가 낮다면-> well diversified, 숫자가 높다면 -> dominant(몰렸다, 독점적이다.)
    - threshold에 따라 efficient frontier 시각화가 어떻게 변화하는지 궁금하다(교수님 의견)


Expected Return 계산

- 채권: (기대)inflation + 실질 수익률(성장) = GDP
- 부동산: (연간)배당수익률
- Index(주식): 1/PER
- Growth(주식): theme 성장률 + alpha(ex: 연 3%로 잡아야 한다(최대한 보수적으로))

